In [5]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch
import torch.nn as nn
from diffusers import StableDiffusionPipeline, DDIMScheduler
from PIL import Image
import numpy as np
from other_attacks import model_transfer
import random
import sys
from natsort import ns, natsorted

In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(254 * 254, 1024)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = x.view(x.shape[0],x.shape[1], x.shape[2]* x.shape[3])
        x = self.fc1(x)
        
        return x

In [13]:
# 假设图像或者视频模态为 input1
input1 = torch.randn(1, 32, 1024).cuda()  # 模态1特征，大小为 [batch_size, num_channels, feature_dim]
input2 = torch.randn(1, 32, 1024).cuda()  # 模态2特征，大小为 [batch_size, num_channels, feature_dim]
input3 = torch.randn(1, 32, 1024).cuda()  # 模态3特征，大小为 [batch_size, num_channels, feature_dim]

outer_product1 = torch.einsum('bci,bcj->bcij', input1, input2)
outer_product2 = torch.einsum('bci,bcj->bcij', input1, input3)

In [14]:
model = ConvNet().cuda()
out1 = model(outer_product1)
out2 = model(outer_product2)

In [15]:
outer_product = torch.einsum('bci,bcj->bcij', out1, out2)
out = model(outer_product)

In [16]:
out.shape

torch.Size([1, 32, 1024])

In [ ]:
# 扩散模型子结构测试

repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

prompt = "Best quality, masterpiece, ultra high res, (photorealistic:1.4), raw photo, 1girl, offshoulder, in the dark, deep shadow, low key, cold light"
image = pipe(prompt, num_inference_steps=50).images[0]
image.save("astronaut.png")

pretrained_diffusion_path = "stabilityai/stable-diffusion-2-base"
ldm_stable = StableDiffusionPipeline.from_pretrained(pretrained_diffusion_path).to('cuda:0')
ldm_stable.scheduler = DDIMScheduler.from_config(ldm_stable.scheduler.config)

code_all = ldm_stable.tokenizer("hello world xxxxxxxxfdgdfgdgxxxxxxxxxasfsdfsfdf")
code = ldm_stable.tokenizer.encode("hello world xxxxxxxxfdgdfgdgxxxxxxxxxasfsdfsfdf")
code_str = ldm_stable.tokenizer.decode(code)

print(code_all)
print(code)
print(code_str)